# 29 - Searching

---

Often you have to "search" for some element in data. Checking all data items to see if the element is available is rather inefficient. Therefore more efficient search methods have been developed, which make use of the structure of data. This chapter discusses several basic search algorithms for Python lists. How special data structures can be used to make searching in data easier, will be discussed in later chapters.

---

## Searching in lists

This chapter focuses on searching in Python lists. Python has implemented several search methods for lists, but since you might want to search in different data structures too, it is good to know how search methods may work. Note that you may assume that accessing elements of lists by their index is a very fast operation for Python, which is `O(1)`. Why that is, will be discussed in the next chapter, in which it is relevant to understand exactly how Python has implemented lists.

The two search methods which Python has implemented for lists are:

- `index()`, which returns the index in the list of the first occurrence of the element that is given as argument
- `count()`, which returns how often the element that is given as argument occurs in the list

There is also the `in` operator (which, you may have learned from the chapter on operator overloading, is actually the `__contains__()` method), which returns `True` if an element is in the list, and `False` otherwise.

### Linear search

If there is nothing known about the contents of the lists, the only available search method is a so-called "linear search", i.e., processing the items all in order until the target element is found or the end of the list is reached. Python has implemented the methods discussed above as linear search methods. For clarity, I will give functions that emulate the behavior of these search methods below:

In [ ]:
def my_index( item, tlist ):
    i = 0
    while i < len( tlist ):
        if tlist[i] == item:
            return i
        i += 1
    return -1

def my_count( item, tlist ):
    count = 0
    i = 0
    while i < len( tlist ):
        if tlist[i] == item:
            count += 1
        i += 1
    return count

def my_in( item, tlist ):
    return my_index( item, tlist ) >= 0

fruitlist = ["apple", "banana", "cherry", "apple", "apple", "peach", "cherry", "apple"]
print( "The index of banana is", my_index( "banana", fruitlist ) )
print( "The count of apple is", my_count( "apple", fruitlist ) )
if my_in( "durian", fruitlist ):
    print( "There is a durian on the list" )
else:
    print( "There is no durian on the list" )

Note that I could have implemented the `my_count()` function by using a `for ... in ...` construction. I did not do that for two reasons: (1) it would then use the reserved word `in`, and I am showing my own implementation of `in`; and (2) in practice, Python processes lists using indices, as that can be done very speedily.

### Time complexity of linear search

The time complexities of the implementations I give above are as follows (you can check):

- `my_index()` is `O(n)` and &Omega;`(1)`
- `my_count()` is `O(n)`, &Omega;`(n)`, and &Theta;`(n)`
- `my_in()` is `O(n)` and &Omega;`(1)`

All linear search methods are `O(n)`. They cannot be anything else, because (in the worst-case scenario) they have to see each and every element once. 

To improve upon this, either the data has to be ordered in a particular way in a list, or a data structure must be designed that supports an efficient way of ordering the data. I will discuss two common search methods which can be used with lists below. More complex data structures that support advanced searching will follow in later chapters.

---

## Binary search

Binary search is a common, efficient method to search in *sorted* lists. The binary search method already came up earlier in the course, but was not named as such. 

### Guessing game

One place where binary search was brought up, was in programming a guessing game. This game works as follows: One player picks a number between 1 and 1000, and the other player tries to guess it. After each number that the second player guesses, the first player says whether the number that he picked is lower, higher, or equal to the guessed number.

A method to guess the number is by starting at 1, and then increasing by 1 after every guess. This method is `O(n)`, as it emulates linear search. However, since the number is picked from a *sorted* list (it is sorted because the first player can say "lower" or "higher" after every guess), it is possible to identify the target number more efficiently. You probably see how, but I will make it explicit:

Suppose that the first player picked 278. The second player first divides the range of numbers 1-1000 into two ranges and one number that is between the ranges, namely: 1-499, 500, and 501-1000. The single number in the middle is used as guess, i.e., 500. The first player says that the target is "lower". This means that now the range of numbers that needs to be searched is 1-499. 

The second player then divides the remaining range again into two (almost) equal parts, and the middle number: 1-249, 250, and 251-499. The guess is 250, and the second player learns that the target is "higher". So now the range to search is 251-499, which is less than a quarter of the original range, after only two guesses.

The guessing continues, each guess dividing the range to search into half of what it was previously. By the time the length of the range is 1, the number has been guessed. This happens in the worst case after `log`<sub>2</sub>`(n)+1` guesses, whereby `n` is the size of the original range. For 1000 numbers, this is 10 guesses at most. 

Note that it does not matter whether a list is used where the distance between each following two items is a constant. When I say that you pick the *middle*, that means you pick the item with the middle *index*.  

### Binary search implementation

The method described above, whereby you divide the range to search in two halves after every trial, is called "binary search". It is `O(log(n))` and &Omega;`(1)`.  

A common implementation of binary search retains the lower and the higher index for the range which is searched. For every trial, the middle of these two indices is calculated and checked. On the basis of this check, either the value for the middle will be returned, or either the lower index or the higher index will be set to the middle plus 1 or minus 1, respectively. Typical code is shown below:

In [ ]:
# This implementation of binary search returns the index of the target in tlist.
# If target is not found in the list, it returns -1.
def binary_search( tlist, target ):
    lo = 0
    hi = len( tlist )-1
    while lo <= hi:
        mid = lo + (hi-lo)//2
        if tlist[mid] == target:
            return mid            
        elif tlist[mid] < target: 
            lo = mid+1
        else:
            hi = mid-1
    return -1

numlist = [-4, -2, -1, 1, 5, 8, 11, 12, 15, 16, 22, 35, 37, 39, 40, 50, 60, 72, 88, 91, 92, 93, 95]
for i in range( 10 ):
    index = binary_search( numlist, i )
    if index >= 0:
        print( "Index of value", i, "in the list is", index )
    else:
        print( "Value", i, "is not in the list" )

---

## Hash tables

Binary search is a huge improvement over linear search. It still requires about `log`<sub>2</sub>`(n)` trials, though. Is it possible to improve upon that?

Suppose that I have a list of names of personnel, and each employee has an employee number that has six digits. I store the employee names in a list, where they are a tuple of the employee number and their name. For example, the list could look like:   

In [ ]:
employees = [
    ( 142351, "Apricot, A." ),
    ( 211111, "Cherry, C." ),
    ( 232112, "Banana, B." ),
    ( 341123, "Fig, F." ),
    ( 400012, "Apple, A." ),
    ( 513210, "Mango, M." ),
    ( 778899, "Pomegranate, P." ), 
    ( 963210, "Pear, P." ) ]


Since the list is sorted on employee number, if I want to search for employees by number, I can use binary search. For these eight employees, it may take up to 4 checks with binary search to find out if a particular number is on the list. If this list would contain 1000 employees, it would take about 10 checks.

__Exercise__: Implement a binary search for the employee list above to search for employees by their number.

Can this be done faster? Can you imagine a way to store the employees in a list in such a way that you can check for the existence of their number with a method that is `O(1)`?

A possible solution would be to store the names of the employees in a list, whereby the index for their name in the list is their employee number. So to find out if employee with number 513210 exists, I just check if `employee[513210]` contains an empty string (in which case no employee exists with this number), or a name. Thus, checking for existence of an employee takes just one attempt.

Naturally, this solution is not realistic in most cases. For six-digit employee numbers, it needs a list with one million elements. While that might still be doable for modern computers, with bigger numbers they might get problems. Moreover, this approach will not work if the list is not sorted by number, but, for instance, by name. A name cannot be  used (directly) as index for a list.

Hopefully at this point you have been thinking "Wouldn't I use a dictionary in that case?" And the answer is, "Of course, you would use a dictionary." A dictionary, however, is no more than a list which is ordered in a very smart way, so that you can use any type of data as index. A dictionary is implemented as a so-called "hash table". 

### Hash table idea

The idea behind hash tables is as follows.

A hash table consists of a list of a specific size. Let's call the size of the list `N`. An element must be stored in the hash table with a particular "key", for instance a string. A so-called "hash function" is used to turn that key into a number (how exactly it does that I will not go into here, just assume that there is a hash function that may turn any string into some number, whereby different strings tends to get different numbers). Let's call this number `H`. The element is then stored in the list at index `H%N` (`H` modulo `N`). 

Consequently, to detect whether an element with a particular key exists in the hash table, you translate that key with the hash function to its associated number `H`, and check whether there is something stored in the hash table at `H%N`.

Of course, there are many issues with this procedure. It is very well possible that two different keys end up being hashed to the same number. It is even more likely that two different hash numbers end up at the same index in the hash table. Moreover, even if it would be guaranteed that all hash numbers translate to different indices, at some point the hash table will be completely filled up, and where are new elements going to be stored then?

The issues mentioned are solved as follows:

- For the problem that two different keys lead to the same hash number, the solution is that in the hash table each element is stored as a tuple with the key. Therefore, when you find an element in the hash table, you can check whether indeed it has the key that you are looking for.
- For the problem that two or more elements might be mapped to the same index in the hash table, the solution is that if an element needs to get stored at an index where already an element is stored (called a "collision"), a so-called "perturbation" value is added to the index to create another index. If that slot is empty, the element gets stored in it. If it is full, repeatedly the perturbation value is added to the index, until an empty slot is found. This is called "probing". 
- For the problem of the hash table getting filled up, the solution is that once the hash table is filled for a particular percentage (e.g., `70%`), a bigger hash table is created and all elements are moved to the new table. Naturally, this can be an expensive operation.

### Hash table procedure

The procedure to store an element in a table with size `N` is therefore as follows:

    if table contains more than FACTOR*N elements:
        grow the table, giving N a new value
    calculate H from the key
    initialize P to 0
    while table[(H+P)%N] is filled:
        if table[(H+P)%N] has the key that is sought:
            overwrite table[(H+P)%N] with the new value
            break
        if table[(H+P)%N] is empty:
            store the key-value pair in table[(H+P)%N]
            break
        P += PERTURB
        
The procedure described above is easy to understand and should work well when suitable values are chosen for `N`, `FACTOR`, and `PERTURB`, and a suitable hash function is created. These guidelines can be followed:

- For `N`, preferably a value is chosen that ensures that the table never needs to grow, while still not being overly large. However, usually there is no knowledge on this in a program. What is essential, though, is that `N` is a prime number. The reason is that, with `N` being a prime number, it is certain that a procedure which seeks new indices using the perturbation value, will never get into a loop whereby it checks indices that it already checked before, unless the hash table is completely filled.
- For `FACTOR`, usually a value around `0.7` is picked. The reason is that this value is high enough to ensure that there aren't too many open slots in the table, while low enough to ensure that in the case of collisions, empty slots are detected within very few tries.
- For `PERTURB`, it does not matter much which value is picked, so usually `1` is used. This is called "linear probing". The disadvantage of linear probing is that probing might run into long sequences of filled slots. To avoid this, more complex methods need to be implemented to deal with collisions, e.g., "double hashing" (at the end of this chapter there is an exercise about that).

As for the hash function, to make it suitable you basically have to know what kind of keys are possible. Preferably, two keys which are quite similar get very different hash numbers. Also, you want hash numbers not to be clustered to much. Creating such a hash function is not easy. For instance, for a string key, adding up the ordinal values of the characters in the string does not make a good hash function. The reason is that short strings will all end up with low hash numbers. Moreover, anagrams will all end up with the same number. 

Python supplies a function `hash()` which you can use to turn objects into hash numbers. This function seems to work well in turning similar strings into quite different numbers.

In [ ]:
print( hash( "bear" ) )
print( hash( "beer" ) )
print( hash( "bare" ) )

### Removing items from a hash table

To remove items from a hash table, you cannot simply seek the item and then make its location in the hash table empty. The reason is that the item might be part of a chain of items that have the same hash number. I will explain this with an example.

First, an element is stored with the key "apple". This element ends up at index 10. Second, an element is stored with the key "banana". This element ends up at the same hash number as "apple", so it gets stored at index 11 (with "linear probing" and a perturbation value of 1). Then, the key "cherry" is used to store an element, which again ends up with the same hash number, and thus gets stored at index 12. 

When "cherry" is searched for, the hash number indicates index 10, where "apple" is stored. Since this is not "cherry", it checks index 11, where "banana" is stored. Since this is still not "cherry", it checks index 12, and finds "cherry". 

Now, suppose that the element with the key "banana" is removed, by making the slot with index 11 empty. Then, when searching for "cherry", the procedure first finds "apple", and then an empty slot, so it concludes that "cherry" is not in the hash table. This is wrong. 

How can this be solved? Should you move all the elements which are later in the chain to earlier places in the chain? No, that is not a good idea; not only is this an expensive procedure, but these elements might actually be part of different chains, so moving them is not allowed. 

The solution is therefore to indicate that the slot where the item is removed from is "empty", but still potentially part of a chain. Commonly, the key is left in the table, but the associated element is removed. This entails that the slot therefore might be overwritten, but is not considered the end of a chain.

### Hash table implementation

Below I give an example implementation of a hash table. In this implementation, I use a fixed size for the hash table. It simply reports that it is full when it reaches a certain threshold. Growing of a hash table is far less instructive than observing how the index for elements is implemented. To determine the hash value, I use the `hash()` function. 

In [ ]:
SIZE = 13 # should be prime
FACTOR = 0.7 # limit to how many slots may be filled
PERTURB = 1 # perturbation value

hash_table = SIZE * [None] 

# Stores an element in the table
def hash_store( hash_table, element, key ):
    hashnum = hash( key )
    while hash_table[hashnum%SIZE] != None:
        if hash_table[hashnum%SIZE][0] == key: # Key already exists and the element is thus overwritten
            hash_table[hashnum%SIZE] = (key, element)
            return
        if hash_table[hashnum%SIZE][1] == None: # Slot had an element which is now deleted, and can be overwritten
            hash_table[hashnum%SIZE] = (key, element)
            return
        hashnum += PERTURB
    if hash_table.count( None ) < (1 - FACTOR) * SIZE:
        print( "Table is considered full,", key, "cannot be stored" )
        return
    hash_table[hashnum%SIZE] = (key, element)

# Removes an element from the table
def hash_remove( hash_table, key ):
    hashnum = hash( key )
    while hash_table[hashnum%SIZE] != None:
        if hash_table[hashnum%SIZE][0] == key:
            hash_table[hashnum%SIZE] = (key, None) # This indicates that the spot is empty, but can be part of a chain
            return
        hashnum += PERTURB
        
# Returns element if key is in table, otherwise None
def hash_find( hash_table, key ):
    hashnum = hash( key )
    while hash_table[hashnum%SIZE] != None:
        if hash_table[hashnum%SIZE][0] == key:
            if hash_table[hashnum%SIZE][1] != None:
                return hash_table[hashnum%SIZE][1]
        hashnum += PERTURB
    return None

fruitlist = ["apple", "banana", "cherry", "durian", "fig", "grape", "lychee", "mango", "orange", "pear", "tangerine"]
for i in range( len( fruitlist ) ): # store as much fruit as possible
    hash_store( hash_table, i+1, fruitlist[i] )
hash_store( hash_table, 99, "pear" ) # overwrite "pear"
hash_remove( hash_table, "apple" ) # remove "apple"
for i in range( len( fruitlist ) ):
    element = hash_find( hash_table, fruitlist[i] )
    if element:
        print( fruitlist[i], "-", element )
    else:
        print( fruitlist[i], "is not in the table" )

### Time complexity of hash tables

The implementation above uses the `count()` method to determine the number of empty slots in the table. This is not a good idea, as `count()` is an expensive operation (it is `O(n)`). However, `count()` can be avoided by keeping track of the number of filled slots (easily achievable by implementing the hash table as a class, which I did not do above so that I would not have to bother about all the overhead methods that using a class brings with it). So I assume that `count()` is not part of the hash table algorithm.

If each hash number that gets calculated is mapped to a unique index in the table, then any element can be found by a straightforward calculation, i.e., in constant time. This is the ideal situation. 

The worst-case scenario is that the hash table is completely filled up, and all the keys get mapped to the same hash number, and the last element of the chain is searched for. In that case, all elements of the table get traversed. This means that finding elements in a hash table is `O(n)` in these circumstances. However, hash tables have been designed to avoid this happening. Not only are they never completely filled, but the hashing is designed to map keys to different indices. Naturally, the more elements in the table, the higher the chance that there is a collision. But even a collision is not that bad: it only becomes bad if too many collisions happen with the same index.

With a proper hash function and a properly designed hash table, storing elements in the table and finding them leads to no collisions, or at worst 2 or 3. In that case, storing and looking up keys are `O(1)`. If you find that you get substantially more than, say, 2 collisions on average, there are two situations to consider:

- The hash table is overly full and needs to be expanded, or
- The hash table is not very full and you need to implement your algorithms better

In practice, therefore, hash tables are considered to be `O(1)` with a proper implementation.

---

## What you learned

In this chapter, you learned about:

- Linear search
- Binary search
- Hash tables

---

## Exercises

### Exercise 29.1

Consider a two-dimensional list, i.e., a list of lists. Each of the lists in the list of list contains numbers, and these numbers are sorted. Moreover, all the numbers in the first list on the list of lists are lower than all the numbers in the second list. And the numbers in the second list are all lower than those in the third list. Etcetera. You may assume that each of the lists on the list of lists contains at least one number. An example you see in the code block below.

Write an efficient search function that determines whether a number occurs anywhere in this list of lists. Also determine the time complexity of this function.

In [ ]:
# Searching in a list of lists.

listoflists = [ [0, 3, 4, 7, 12, 16, 17, 19],
  [25, 39, 40, 41, 42],
  [50, 51, 56, 57, 58, 72, 81, 82],
  [83, 84, 106, 203, 213, 233, 243, 253, 263, 273, 283],
  [512, 1024, 2048, 4096],
  [4097],
  [5000, 50000, 500000, 678345, 765432, 765433, 800000, 810000, 811000, 811100, 811110, 888888]
]


### Exercise 29.2

Better than linear probing, is using a second hash function for determining the perturbation value for a hash table. This is called "double hashing". The advantage is that collisions whereby different keys end up at the same index, use different chains to seek for empty slots. The second hash function, which is only used to determine the perturbation value, can usually be simpler than the one that determines the hash number. A possible second hash number for strings is to take the ordinal values for the first and the last character of the string, and add them up. You just have to make sure that the perturbation value, when taken modulo the size of the table, is not zero, otherwise you will get into endless loops. Change the example implementation that I give above to encompass double hashing.

In [ ]:
# Hash table with double hashing.


---

End of Chapter 29. Version 1.1. 